In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

import shap
import warnings
warnings.filterwarnings('ignore')


In [52]:
# Set dataset path
data_dir = "PPMI_ds/"

### --- Clinical + Cognitive Modality --- ###
clinical_files = {
    "diagnosis": "Primary_Clinical_Diagnosis_18Mar2025.csv",
    "parkinsonism": "Features_of_Parkinsonism_18Mar2025.csv",
    "rem": "Features_of_REM_Behavior_Disorder_18Mar2025.csv",
    "cgi": "Clinical_Global_Impression__CGI__-_Investigator_18Mar2025.csv",
    "exam": "General_Physical_Exam_18Mar2025.csv"
}

clinical_dfs = {name: pd.read_csv(os.path.join(data_dir, fname)) for name, fname in clinical_files.items()}

# General Physical Exam (pivoted)
exam_df = clinical_dfs['exam'][['PATNO', 'PECAT', 'ABNORM']]
exam_wide = exam_df.pivot_table(index='PATNO', columns='PECAT', values='ABNORM', aggfunc='max')
exam_wide.columns = [f"ABNORM_{str(col).strip().replace(' ', '_')}" for col in exam_wide.columns]
exam_wide.reset_index(inplace=True)

# Diagnosis and label
label_df = clinical_dfs['diagnosis'][['PATNO', 'PRIMDIAG']].drop_duplicates(subset='PATNO')
label_df['label'] = label_df['PRIMDIAG'].apply(lambda x: 1 if x == 17 else 0)
label_df = label_df[['PATNO', 'label']]

# Aggregate cognitive/clinical data
aggregated = []
for key in ['parkinsonism', 'rem', 'cgi']:
    df = clinical_dfs[key].drop(columns=[
        'EVENT_ID', 'INFODT', 'REC_ID', 'PAG_NAME', 'ORIG_ENTRY', 'LAST_UPDATE'
    ], errors='ignore')
    aggregated.append(df.groupby('PATNO').mean(numeric_only=True).reset_index())

# Merge clinical features
clinical_merged = label_df.copy()
for df in aggregated + [exam_wide]:
    clinical_merged = clinical_merged.merge(df, on='PATNO', how='left')

### --- Genetic + Biomarker Modality --- ###
genetic_df = pd.read_csv(os.path.join(data_dir, "iu_genetic_consensus_20250225_18Mar2025.csv"))
variants_df = pd.read_csv(os.path.join(data_dir, "PPMI_PD_Variants_Genetic_Status_WGS_20180921.csv"))
sbr_df = pd.read_csv(os.path.join(data_dir, "DaTScan_SBR_Analysis_18Mar2025.csv"))
diagnosis_df = pd.read_csv(os.path.join(data_dir, "Primary_Clinical_Diagnosis_18Mar2025.csv"))

# Merge all genetic tables
genetic_merged = genetic_df.merge(variants_df, on='PATNO', how='inner')
genetic_merged = genetic_merged.merge(sbr_df, on='PATNO', how='inner')
genetic_merged = genetic_merged.merge(diagnosis_df[['PATNO', 'PRIMDIAG']], on='PATNO', how='inner')

# Clean up columns with excessive missingness and convert binaries
genetic_merged = genetic_merged.dropna(axis=1, thresh=len(genetic_merged) * 0.6)
binary_cols = ['CLIA', 'GWAS', 'WES', 'WGS', 'SVs', 'SANGER', 'IU_Fingerprint', 'RNASEQ', 'RNASEQ_VIS']
for col in binary_cols:
    if col in genetic_merged.columns:
        genetic_merged[col] = genetic_merged[col].map({'X': 1, '-': 0})

# Encode categorical columns
non_numeric = genetic_merged.select_dtypes(include=['object']).columns
encoder = OrdinalEncoder()
genetic_merged[non_numeric] = encoder.fit_transform(genetic_merged[non_numeric])

# Retain PATNO and drop duplicate PRIMDIAG
genetic_merged = genetic_merged.drop(columns=['PRIMDIAG'], errors='ignore')

### --- Merge Both Modalities on PATNO --- ###
final_df = clinical_merged.merge(genetic_merged, on='PATNO', how='inner')

# Final shape and export

final_df = final_df.drop_duplicates()


df_sorted = final_df.sort_values(by=['PATNO', 'DATSCAN_DATE'])  # or 'EVENT_ID'
df_deduped = df_sorted.drop_duplicates(subset='PATNO', keep='first')

leakage_cols = ['PSGLVL'] + [col for col in df_deduped.columns if 'DATSCAN' in col]
df_deduped = df_deduped.drop(columns=leakage_cols)

print("✅ Final combined dataset shape:", df_deduped.shape)
df_deduped.to_csv("PPMI_multimodal_combined.csv", index=False)


✅ Final combined dataset shape: (757, 114)


In [53]:
# Load combined dataset
df = pd.read_csv("PPMI_multimodal_combined.csv")

# Check shape and head
print("Dataset shape:", df.shape)
df.head()

print(list(df.columns))

Dataset shape: (757, 114)
['PATNO', 'label', 'FEATBRADY', 'FEATPOSINS', 'FEATRIGID', 'FEATTREMOR', 'RBDDIAG', 'RBDPSG', 'INVESTAST', 'ABNORM_Abdomen', 'ABNORM_Cardiovascular_(including_peripheral_vascular)', 'ABNORM_Ears/Nose/Throat', 'ABNORM_Eyes', 'ABNORM_Head/Neck/Lymphatic', 'ABNORM_Lungs', 'ABNORM_Musculoskeletal', 'ABNORM_Neurological', 'ABNORM_Other', 'ABNORM_Other_(Specify_location_and_describe)', 'ABNORM_Psychiatric', 'ABNORM_Skin', 'CLIA', 'GWAS', 'WES', 'WGS', 'SVs', 'SANGER', 'IU_Fingerprint', 'RNASEQ', 'RNASEQ_VIS', 'APOE', 'PATHVAR_COUNT', 'VAR_GENE', 'LRRK2', 'GBA', 'VPS35', 'SNCA', 'PRKN', 'PARK7', 'PINK1', 'chr1:154925709:G:C_C_PMVK_rs114138760', 'chr1:155235252:A:G_G_GBA_L444P_rs421016', 'chr1:155235843:T:C_C_GBA_N370S_rs76763715', 'chr1:155236246:G:A_A_GBA_T408M_rs75548401', 'chr1:155236376:C:T_T_GBA_E365K_rs2230288', 'chr1:155240629:C:T_T_GBA_IVS2+1_rs104886460', 'chr1:155240660:G:GC_GC_GBA_84GG_rs387906315', 'chr1:205754444:C:T_C_NUCKS1_rs823118', 'chr1:226728377:T

In [54]:
# Define target and features
y = df['label']
X = df.drop(columns=['PATNO', 'label'])

# Drop columns with all missing values (or >50% missing if you prefer)
missing_threshold = 1.0  # use 0.5 for 50% missingness threshold
X = X.loc[:, X.isnull().mean() < missing_threshold]

print("Remaining columns after dropping all-NaN cols:", X.shape[1])


# Impute missing values with KNN
imputer = KNNImputer(n_neighbors=5)
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Check for any remaining missing values
print("Remaining NaNs:", X_imputed.isna().sum().sum())


Remaining columns after dropping all-NaN cols: 108
Remaining NaNs: 0


In [55]:
# Print original class distribution
print("Original class distribution:", Counter(y))

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_imputed, y)

print("Resampled class distribution:", Counter(y_resampled))


Original class distribution: Counter({0: 566, 1: 191})
Resampled class distribution: Counter({1: 566, 0: 566})


In [58]:
rf_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'rf_importance': rf.feature_importances_
}).sort_values(by='rf_importance', ascending=False)

print(rf_importance_df.head(10))

                                 feature  rf_importance
0                              FEATBRADY       0.234270
2                              FEATRIGID       0.150661
1                             FEATPOSINS       0.133246
3                             FEATTREMOR       0.083607
12                   ABNORM_Neurological       0.047872
4                              INVESTAST       0.015265
54  chr4:958159:T:C_C_TMEM175_rs34311866       0.013500
25                         PATHVAR_COUNT       0.012834
20                                   SVs       0.011475
26                              VAR_GENE       0.010852


In [57]:
# Train-test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
)

# Train a random forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

# Predict
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

# Evaluate
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_proba))


Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       170
           1       0.94      0.97      0.95       170

    accuracy                           0.95       340
   macro avg       0.95      0.95      0.95       340
weighted avg       0.95      0.95      0.95       340

ROC-AUC Score: 0.9913840830449827


In [49]:
import shap
import numpy as np
import pandas as pd

# Use TreeExplainer with the raw model and no background data
explainer = shap.TreeExplainer(rf)

# This returns a list of arrays, one for each class
shap_values = explainer.shap_values(X_test)

# Get SHAP values for class 1
shap_values_class_1 = shap_values[1]

# Make sure it's aligned with X_test
mean_abs = np.abs(shap_values_class_1).mean(axis=0)

# Truncate or align just in case
min_len = min(len(mean_abs), len(X_test.columns))

shap_importance_df = pd.DataFrame({
    'feature': X_test.columns[:min_len],
    'mean_abs_shap': mean_abs[:min_len]
}).sort_values(by='mean_abs_shap', ascending=False).reset_index(drop=True)

# Show top features
print(shap_importance_df.head(20))




      feature  mean_abs_shap
0   FEATBRADY       0.004452
1  FEATPOSINS       0.004452
